# Understand the life cycle of floorplans
- DS: Nhan Le
- PM: Sean Corriel

**Objective: **

    - how often are 3D floor plan users creating new floor plans over time

    - what frequency buckets do customers fall into? 

    - how is this distributed across customer base

    - how does this behavior differ between free trial / paid pros

    - how we can we distinguish (via dashboards) number of floor plans created by free trial vs paid customers

 

In Future phases, we can enrich this understanding with engagement events - editing properties within a floor plan, to gain a more resolution into the lifecycle of floor plans

adding a 3D product

modifying a 3D object (wall, or product) 

Sharing a 3D Floor Plan

Floor Plan Creation on mobile or web

Floor Plan Duplication - duplicated a floor plan and applying ‘edits’ to a duplicated floor planNeeded for access to all other apps. It would be a good idea to install ASAP. 

In [1]:
from pyhive import presto
import pandas as pd
import matplotlib as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

from pyhive import presto
conn = presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086)

In [21]:
def validate_floorplans():
    # sample data from floorplan elements
    # extract floorplans with company_user_id, created_dt
    # every floorplan has 9 (distinct) elements; must have at least one more to be considered "real"
    valid_fp_sql = '''
    -- get valid floorplans
    fp as (
        select floorplan_id, company_user_id, created_dt, created_at
            , count(floorplan_element_id) over (partition by floorplan_id) as fp_elements
            , min(date(created_dt)) over (partition by floorplan_id) as fp_created_dt 
            -- get 1 row per flooplan id
            , row_number() over (partition by floorplan_id) as row_num
        from gaia.dim_floorplan_elements 
        )
    , fp1 as (select floorplan_id, company_user_id, created_dt, fp_created_dt, fp_elements
        , case when fp_elements > 9 then 'valid' else 'invalid' end as fp_status
        , coalesce(date_diff('minute', lag(created_at) over(partition by company_user_id order by created_at), created_at), 2) as minutes_since_prev
    from fp
    where fp_elements > 9
    and row_num = 1
    order by floorplan_id, fp_created_dt
    )

    -- exclude floorplans within 1 minute of previous one
    , val_fp as (
        select * from fp1 where minutes_since_prev >1
        )

    -- get company info
    -- need to worry about renewals
    '''
    return valid_fp_sql
pd.read_sql('with ' + validate_floorplans() + 'select * from val_fp limit 5' , conn)

,floorplan_id,company_user_id,created_dt,fp_created_dt,fp_elements,fp_status,minutes_since_prev
0,152469592,210249,2020-01-16,2019-12-10,46,valid,38423
1,152498473,57045505,2019-12-11,2019-12-11,20,valid,2
2,152500949,57045505,2019-12-11,2019-12-11,89,valid,89
3,152501711,57045505,2019-12-11,2019-12-11,63,valid,72
4,152736076,16821960,2019-12-17,2019-12-17,30,valid,2


In [23]:
def get_dim_companies(time_frame = '2020-01-01'): 
    # obtain country, pro_type, zip, metro area from dim_company_users
    # obtain sku, join_date from dim_company_user_sku_ranges
    # join_date is the first day of the month when the company first became paying customers
    dim_comp_sql = '''
    dim_comp as (
        select company_user_id, join_date 
        from (
            select company_user_id, start_dt as join_date 
            from (
                -- dim_company_user_sku_ranges provides start_dt, country, sku, and free_or_paying
                -- dim_company_users has ivy_company_type and is_likely_houzzer 
                select  s.company_user_id
                    , s.start_dt 
                    , account_created_date
                    , row_number() over(PARTITION BY s.company_user_id
                                        ORDER BY start_dt ASC) AS row_num
                from gaia.dim_company_user_sku_ranges s 
                join gaia.dim_company_users c 
                on s.company_user_id = c.company_user_id
                where free_or_paying = 'paying'
                    and NOT c.is_likely_houzzer 
                    and c.ivy_company_type <> 'Not in ivy'
                )a
            where row_num = 1
            )b
        where date(join_date) >= date('{}')
    )

    -- get ft_date as first day in free trial status
    , ft as ( 
        select company_user_id, sku, country, pro_type, first_start_date, zip, metro_area, free_or_paying 
        from (
            select *,  start_dt as first_start_date
            from (
                select  s.*, zip, metro_area
                    --s.company_user_id, 
                    --start_dt,
                    --sku,
                    , case when country = 'AU' then 'AU'
                        when country = 'CA' then 'CA'
                        when country = 'DE' then 'DE'
                        when country = 'ES' then 'ES'
                        when country = 'FR' then 'FR'
                        when country = 'GB' then 'GB'
                        when country = 'IE' then 'IE'
                        when country = 'IT' then 'IT'
                        when country = 'JP' then 'JP'
                        when country = 'NZ' then 'NZ'
                        when country = 'RU' then 'RU'
                        when country = 'US' then 'US'
                        else 'Other' end as country

                    , ivy_company_type as pro_type 
                    , row_number() over(PARTITION BY s.company_user_id
                                        ORDER BY start_dt ASC) AS row_num
                            
                from gaia.dim_company_user_sku_ranges s 
                join gaia.dim_company_users c 
                on s.company_user_id = c.company_user_id
                where --free_or_paying <> 'paying' and 
                        NOT c.is_likely_houzzer 
                        and c.ivy_company_type <> 'Not in ivy'
                )a
        where row_num = 1
        )b
    )

    -- most recent is_still_live and sku status
    , live as (
        select company_user_id, is_still_live, sku as most_recent_sku
        from (  
            select *
            , row_number() over(PARTITION BY company_user_id ORDER BY end_dt DESC) AS row_num
            from gaia.dim_company_user_sku_ranges
            )a
        where row_num = 1
        )

    -- if paying customers are no longer live, get their last live date
    , last_live as (
        select company_user_id, end_dt as last_live_date
        from (  
            select *
            , row_number() over(PARTITION BY company_user_id ORDER BY end_dt DESC) AS row_num
            from gaia.dim_company_user_sku_ranges
            where free_or_paying = 'paying'
            )a
        where row_num = 1
        )        
    -- distinguish between companies who ever did free try
    ,first as (
        select ft.company_user_id, ft.country, ft.first_start_date, comp.join_date, ft.pro_type, ft.zip, ft.metro_area
            , case when first_start_date >= join_date then 'joined' else free_or_paying end as first_status
            , is_still_live, last_live_date, most_recent_sku

            -- pre paying days
            , date_diff('day', date(first_start_date), date(join_date)) as pre_paying_days

            -- activated days: use last live date if not live, othwerwise use today
            --, date_diff('day', date(join_date),
            --    case when is_still_live = 1 then current_date
            --    else date(last_live_date) end) as survived_days
        from ft -- include both ft and never-joined companies
        left join dim_comp comp 
        on comp.company_user_id = ft.company_user_id
        left join live
        on live.company_user_id = ft.company_user_id
        left join last_live
        on last_live.company_user_id = ft.company_user_id
        
    )    

    
    '''.format(time_frame)
    return dim_comp_sql


In [32]:
# get sample 5 rows from get_dim_companies
pd.read_sql('with ' + get_dim_companies() + 'select * from first  limit 5', conn)

,company_user_id,country,first_start_date,join_date,pro_type,zip,metro_area,first_status,is_still_live,last_live_date,most_recent_sku,pre_paying_days
0,151316,CA,2021-08-26,None,Others,E1J1J4,~other metros,freemium,0,None,Freemium 1.0,NaN
1,556097,GB,2020-01-01,2020-01-01,Designers,SN10 2GH,None,joined,0,2020-08-20,Freemium 1.0,0.0
2,9996171,US,2021-08-26,None,Others,02842,providence,freemium,0,None,Freemium 1.0,NaN
3,13029138,Other,2021-08-26,None,Builders,9000,~other metros,freemium,0,None,Freemium 1.0,NaN
4,16177214,US,2021-08-26,None,Others,07304,new york,freemium,0,None,Freemium 1.0,NaN


In [30]:
# get number of valid floorplans over time and plot by company paying status
def join_val_fp_companies():
    sql = 'with ' + validate_floorplans() + ', ' + get_dim_companies() + '''
    -- join val_fp and first by company_user_id
    -- floor status = free if its date was before join_date or after last_live_date
    -- TO VERIFY: floor status = paying if its date was between join_date and last_live_date
    ,df as (
        select 
            first.*
            , floorplan_id, fp_created_dt, fp_status, minutes_since_prev
            , case 
                when date(fp_created_dt) between date_add('day', -30, date(join_date)) and date(join_date) then  '1:free trial (converted)'
                when date(fp_created_dt) between date(join_date) and date_add('day', 30, date(join_date)) then  '2:0-30d payment'
                when date(fp_created_dt) between date_add('day', 31, date(join_date)) and date_add('day', 90, date(join_date)) then  '3:30-90d payment'
                when date(fp_created_dt) between date_add('day', 91, date(join_date)) and date_add('day', 365, date(join_date)) then  '4:90d-1yr payment'
                when date(fp_created_dt) between date_add('day', 366, date(join_date)) and date(last_live_date) then  '5:1y+ payment'
                else '0:free' end as fp_paying_status
        from first
        left join val_fp
        on first.company_user_id = val_fp.company_user_id
        where fp_status = 'valid'
            --and country = 'US'
        order by first.company_user_id, fp_created_dt
    )
    '''
    return sql


In [31]:
# sample the first 20 rows of join_val_fp_companies
pd.read_sql(join_val_fp_companies() + 'select * from df limit 20', conn)

,company_user_id,country,first_start_date,join_date,pro_type,zip,metro_area,first_status,is_still_live,last_live_date,most_recent_sku,pre_paying_days,floorplan_id,fp_created_dt,fp_status,minutes_since_prev,fp_paying_status
0,1536,US,2020-09-23,2020-09-23,Snail Cage,13090,new york,joined,1,2022-10-13,Ultimate,0.0,164821150,2020-10-01,valid,2,2:0-30d payment
1,1565,US,2020-01-01,2020-01-01,Snail Cage,94549,san francisco,joined,1,2022-10-13,Ultimate,0.0,163061261,2020-08-13,valid,2,4:90d-1yr payment
2,1565,US,2020-01-01,2020-01-01,Snail Cage,94549,san francisco,joined,1,2022-10-13,Ultimate,0.0,189800052,2022-08-29,valid,1074330,5:1y+ payment
3,4218,US,2021-08-26,2022-04-29,Designers,78704,austin,freemium,1,2022-10-13,Ultimate,246.0,186005175,2022-04-28,valid,2,1:free trial (converted)
4,4263,US,2022-10-02,None,Designers,82834,~other metros,freemium,0,None,Paying Ent. Without SFDC Sku,NaN,190811303,2022-10-02,valid,2,0:free
5,5461,US,2020-01-01,2020-01-01,Snail Cage,85254,phoenix,joined,1,2022-10-13,Ultimate,0.0,185168037,2022-03-30,valid,2,5:1y+ payment
6,5677,US,2021-08-26,None,Snail Cage,19002,philadelphia,freemium,0,None,Freemium 1.0,NaN,164517915,2020-09-23,valid,2,0:free
7,6064,US,2021-01-27,2021-02-26,Snail Cage,94025,san francisco,free trial,1,2022-10-13,Starter,30.0,168981949,2021-01-27,valid,2,1:free trial (converted)
8,6064,US,2021-01-27,2021-02-26,Snail Cage,94025,san francisco,free trial,1,2022-10-13,Starter,30.0,169069400,2021-01-29,valid,2534,1:free trial (converted)
9,6064,US,2021-01-27,2021-02-26,Snail Cage,94025,san francisco,free trial,1,2022-10-13,Starter,30.0,169244034,2021-02-02,valid,5553,1:free trial (converted)
